In [1]:
#调用接口
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorlayer as tl
from TransferLearningWrapV1 import TransferLearningMethod

In [2]:
#全局变量
IDcol = 'jdpin'
label ='card_desire'
file_dir = './'
pipeline_model = 'data_precessed_pipeline_{}.pkl'.format('Mscale')  #数据预处理pipeline
encoder_model = file_dir + 'model2/stack_encoder_lab_{}.ckpt'.format('Mscale') #堆栈式编码器模型
encoder_cols = ['jdmall_user_f0002','jrlab_up_career_value']
new_features = False
model_type = 'MediumParity'
# new_num = 
transfer_learning_model = file_dir+ 'model2/transfer_learning_lab_{}.ckpt'.format(label)

In [3]:
def get_test_data(data_name):
    from preprocessing import DataProcessing
    dp = DataProcessing(['pin']) 
    data = pd.read_csv('./data/{}.csv'.format(data_name))
    y = data.loc[:,data_name]
    cols=['jrlab_up_career_value']
    data_X = data.loc[:,cols]
    data_X.replace(['a','Z'],-1,inplace=True)
    values = {'jrlab_up_pay_f0036': '2000-01-01', 'jrlab_up_pay_f0037':'2000-01-01', 'jdmall_ordr_f0001':'2000-01-01', 'jdmall_ordr_f0002':'2000-01-01'}
    data_X.fillna(value=values,inplace=True)
    # 时间类变量
    time_vars = ['jrlab_up_pay_f0036','jrlab_up_pay_f0037','jdmall_ordr_f0001']
    #异常时间数据处理
    for var in time_vars:
        data_X.loc[:,var] = data_X.loc[:,var].str[:10]
    # data_time = data.loc[:,time_vars]
    time_vars = ['jrlab_up_pay_f0036','jrlab_up_pay_f0037','jdmall_ordr_f0001']
    data_X = dp.deal_with_time_variables(data_X,time_vars, end_date='2018-11-17')
    data_X = data_X.fillna(-1)
    data_precessed_pipeline = joblib.load('data_precessed_pipeline_Mscale .pkl')
    data_X_scale = data_precessed_pipeline.transform(data_X)
    data_scale = pd.DataFrame(data_X_scale)
    data_scale.columns = cols
    data_scale[data_name] = y.values
    return data_scale

In [4]:
data = get_test_data(label)

--> RUN TIME: <deal_with_time_variables> : 0.015638351440429688


In [5]:
data['test_col1'] = 1
data['test_col2'] = 2

In [6]:
# 实例化迁移学习方法库
tlm = TransferLearningMethod(data, None, 'card_desire', encoder_cols)

In [7]:
#加载模型
act = tf.nn.relu
x = tf.placeholder(tf.float32, shape=[None, 396], name='x')
network = tl.layers.InputLayer(x, name='input_layer')
network = tl.layers.DropoutLayer(network, keep=0.8, name='denoising1')
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, n_units=200, act=act, name= 'relu1')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, n_units=100, act=act, name= 'relu2')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
network = tl.layers.DenseLayer(network, n_units=50, act=act, name='relu3')
#重载网络
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, save_path = encoder_model)

[TL] InputLayer  input_layer: (?, 396)
[TL] DropoutLayer denoising1: keep: 0.800000 is_fix: False
[TL] DropoutLayer drop1: keep: 0.800000 is_fix: False
[TL] DenseLayer  relu1: 200 relu
[TL] DropoutLayer drop2: keep: 0.500000 is_fix: False
[TL] DenseLayer  relu2: 100 relu
[TL] DropoutLayer drop3: keep: 0.500000 is_fix: False
[TL] DenseLayer  relu3: 50 relu
INFO:tensorflow:Restoring parameters from ./model2/stack_encoder_lab_Mscale.ckpt


In [ ]:
X = data.loc[:,encoder_cols].values

In [ ]:
X.shape

In [8]:
layer_out = tlm.get_layer_outputs(network, sess, x, layer_name='output')

[TL] DenseLayer  output: 1 No Activation
[TL]   [*] geting layers with output
[TL]   got   0: output/bias_add:0   (?, 1)


In [8]:
dev_pred = tlm.retrain_classifier_nets(network, sess, x )

匹配数据量： 2949
所传数据包含2个编码器外新特征...
--> 迁移学习类型：异构型迁移学习
[TL] DenseLayer  output: 1 No Activation
Epoch 1 of 100 took 0.168949s
   train loss: 1.294516
   val loss: 1.695792
Epoch 10 of 100 took 0.079895s
   train loss: 0.598021
   val loss: 0.597826
Epoch 20 of 100 took 0.080642s
   train loss: 0.555636
   val loss: 0.517679
Epoch 30 of 100 took 0.079394s
   train loss: 0.496538
   val loss: 0.484515
Epoch 40 of 100 took 0.079707s
   train loss: 0.394006
   val loss: 0.409839
Epoch 50 of 100 took 0.078919s
   train loss: 0.373364
   val loss: 0.383667
Epoch 60 of 100 took 0.082660s
   train loss: 0.364104
   val loss: 0.386202
Epoch 70 of 100 took 0.079977s
   train loss: 0.344017
   val loss: 0.352157
Epoch 80 of 100 took 0.081363s
   train loss: 0.403222
   val loss: 0.431726
Epoch 90 of 100 took 0.080626s
   train loss: 0.376754
   val loss: 0.405016
Epoch 100 of 100 took 0.079565s
   train loss: 0.368725
   val loss: 0.394677
Evaluation
   test loss: 0.380962
Model saved in file: ./model